In [22]:
from snowflake.snowpark import Session

from snowflake.snowpark.functions import col, to_date, when, sum

from configparser import ConfigParser

from datetime import date, timedelta

import pandas as pd

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

In [40]:
ordenes = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDETALLEORDENES4')
    .with_column('FECHA', to_date(col('FDFECHAHORA')))
    .filter(col('FNIMPTOTAL') > 0)
    .select(
        [
            when(col('FNIMPTOTAL') <= 200, '$0-$200')
            .when(col('FNIMPTOTAL') <= 400, '$201-$400')
            .when(col('FNIMPTOTAL') <= 600, '$401-$600')
            .when(col('FNIMPTOTAL') <= 800, '$601-$800')

            .when(col('FNIMPTOTAL') <= 1000, '$801-$1000')
            .when(col('FNIMPTOTAL') <= 1200, '$1001-$1200')
            .when(col('FNIMPTOTAL') <= 1400, '$1201-$1400')
            .when(col('FNIMPTOTAL') <= 1600, '$1401-$1600')
            .when(col('FNIMPTOTAL') <= 1800, '$1601-$1800')

            .when(col('FNIMPTOTAL') <= 2000, '$1801-$2000')
            .when(col('FNIMPTOTAL') <= 2200, '$2001-$2200')
            .when(col('FNIMPTOTAL') <= 2400, '$2201-$2400')
            .when(col('FNIMPTOTAL') <= 2600, '$2401-$2600')
            .when(col('FNIMPTOTAL') <= 2800, '$2601-$2800')

            .when(col('FNIMPTOTAL') <= 3000, '$2801-$3000')
            .when(col('FNIMPTOTAL') <= 3200, '$3001-$3200')
            .when(col('FNIMPTOTAL') <= 3400, '$3201-$3400')
            .when(col('FNIMPTOTAL') <= 3600, '$3401-$3600')
            .when(col('FNIMPTOTAL') <= 3800, '$3601-$3800')

            .when(col('FNIMPTOTAL') <= 4000, '$3801-$4000')
            .when(col('FNIMPTOTAL') <= 4200, '$4001-$4200')
            .when(col('FNIMPTOTAL') <= 4400, '$4201-$4400')
            .when(col('FNIMPTOTAL') <= 4600, '$4401-$4600')
            .when(col('FNIMPTOTAL') <= 4800, '$4601-$4800')

            .when(col('FNIMPTOTAL') <= 5000, '$4801-$5000')
            .when(col('FNIMPTOTAL') <= 5200, '$5001-$5200')
            .when(col('FNIMPTOTAL') <= 5400, '$5201-$5400')
            .when(col('FNIMPTOTAL') <= 5600, '$5401-$5600')
            .when(col('FNIMPTOTAL') <= 5800, '$5601-$5800')
            .when(col('FNIMPTOTAL') <= 6000, '$5801-$6000')

            .when(col('FNIMPTOTAL') <= 6200, '$6001-$6200')
            .when(col('FNIMPTOTAL') <= 6400, '$6201-$6400')
            .when(col('FNIMPTOTAL') <= 6600, '$6401-$6600')
            .when(col('FNIMPTOTAL') <= 6800, '$6601-$6800')

            .when(col('FNIMPTOTAL') <= 7000, '$6801-$7000')
            .when(col('FNIMPTOTAL') <= 7200, '$7001-$7200')
            .when(col('FNIMPTOTAL') <= 7400, '$7201-$7400')
            .when(col('FNIMPTOTAL') <= 7600, '$7401-$7600')
            .when(col('FNIMPTOTAL') <= 7800, '$7601-$7800')

            .when(col('FNIMPTOTAL') <= 8000, '$7801-$8000')
            .when(col('FNIMPTOTAL') <= 8200, '$8001-$8200')
            .when(col('FNIMPTOTAL') <= 8400, '$8201-$8400')
            .when(col('FNIMPTOTAL') <= 8600, '$8401-$8600')
            .when(col('FNIMPTOTAL') <= 8800, '$8601-$8800')

            .when(col('FNIMPTOTAL') <= 9000, '$8801-$9000')
            .when(col('FNIMPTOTAL') <= 9200, '$9001-$9200')
            .when(col('FNIMPTOTAL') <= 9400, '$9201-$9400')
            .when(col('FNIMPTOTAL') <= 9600, '$9401-$9600')
            .when(col('FNIMPTOTAL') <= 9800, '$9601-$9800')

            .when(col('FNIMPTOTAL') <= 10000, '$9801-$10000')
            .else_('$10001-Resto')
            .alias('BRACKET'),
            'FECHA',
            (col('FNIMPTOTAL') / 1.16).alias('VENTA'),
            'FIIDMARCA'
        ]
    )
)

marcas = session.table('SEGMENT_EVENTS.ALMODCOMUN.TACMARCAS')

tiempo = session.table('WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME')

ordenes_por_anio_y_marca = (
    ordenes
    .join(tiempo, on='FECHA')
    .join(marcas, on='FIIDMARCA')
    .filter(col('ANIO_ALSEA') == 2023)
    .filter(~col('FCNOMBRE').isin(['Cedis', 'La Finca', 'California Pizza Kitchen', 'El Portón']))
    .with_column('MARCA', when(col('FCNOMBRE') == "Domino's Pizza Franquicias", "Domino's Pizza").else_(col('FCNOMBRE')))
    .group_by(['ANIO_ALSEA', 'MES_ALSEA', 'MARCA', 'BRACKET'])
    .agg(sum(col('VENTA')).alias('VENTA'))
)

pd_df = ordenes_por_anio_y_marca.order_by(['MARCA', 'BRACKET']).to_pandas()

brackets = pd.read_csv('brackets.csv')

result = pd.merge(pd_df, brackets, on='BRACKET')

result['RATIO'] = result.groupby(['MARCA', 'MES_ALSEA'])['VENTA'].transform(lambda x: x / x.sum())

result.to_csv('prueba.csv', index = False)